In [14]:
import pandas as pd


train = pd.read_csv('train_ctrUa4K.csv')
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [15]:
train.isnull().sum()
train = train.dropna()
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [16]:
train['Gender']= train['Gender'].map({'Male':0, 'Female':1})
train['Married']= train['Married'].map({'No':0, 'Yes':1})
train['Loan_Status']= train['Loan_Status'].map({'N':0, 'Y':1})

In [17]:
X = train[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount', 'Credit_History']]
y = train.Loan_Status
X.shape, y.shape

((480, 5), (480,))

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size = 0.2, random_state = 10)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Define hyperparameters for tuning
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize Decision Tree
dt = DecisionTreeClassifier(random_state=42)

# GridSearchCV
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Best Model
best_dt = grid_search.best_estimator_


Best Parameters: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}


In [20]:
from sklearn.metrics import accuracy_score, classification_report

# Train Decision Tree with best parameters
best_dt.fit(x_train, y_train)

# Predictions
y_pred_dt = best_dt.predict(x_cv)

# Evaluation
print("Decision Tree Accuracy:", accuracy_score(y_cv, y_pred_dt))
print("Decision Tree Classification Report:\n", classification_report(y_cv, y_pred_dt))


Decision Tree Accuracy: 0.8020833333333334
Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.34      0.51        29
           1       0.78      1.00      0.88        67

    accuracy                           0.80        96
   macro avg       0.89      0.67      0.69        96
weighted avg       0.85      0.80      0.77        96



In [21]:
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train the model
rf.fit(x_train, y_train)

# Predictions
y_pred_rf = rf.predict(x_cv)

# Evaluation
print("Random Forest Accuracy:", accuracy_score(y_cv, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_cv, y_pred_rf))


Random Forest Accuracy: 0.8229166666666666
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.45      0.60        29
           1       0.80      0.99      0.89        67

    accuracy                           0.82        96
   macro avg       0.87      0.72      0.75        96
weighted avg       0.84      0.82      0.80        96



In [22]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=4, random_state = 10)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=4, random_state=10)

In [23]:
from sklearn.metrics import accuracy_score
pred_cv = model.predict(x_cv)
accuracy_score(y_cv,pred_cv)

0.8020833333333334

In [24]:
pred_train = model.predict(x_train)
accuracy_score(y_train,pred_train)

0.8203125

In [25]:
# saving the model
import pickle
pickle_out = open("classifier.pkl", mode = "wb")
pickle.dump(model, pickle_out)
pickle_out.close()